# Chapter 5: Modules and Interfaces

## Module Basics

Modules are the fundamental building blocks of SystemVerilog designs. They encapsulate functionality and provide a way to create hierarchical designs through instantiation.

### Basic Module Structure

```systemverilog
module module_name #(
    // Parameters (optional)
    parameter int WIDTH = 8
) (
    // Port declarations
    input  logic clk,
    input  logic reset_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

    // Module body - internal logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n)
            data_out <= '0;
        else
            data_out <= data_in;
    end

endmodule
```

### Module Instantiation

```systemverilog
// Named port connections (recommended)
module_name #(.WIDTH(16)) inst_name (
    .clk(system_clk),
    .reset_n(sys_reset),
    .data_in(input_data),
    .data_out(output_data)
);

// Positional port connections (not recommended for complex modules)
module_name #(16) inst_name (system_clk, sys_reset, input_data, output_data);
```

### Key Module Concepts

**Scope and Hierarchy**: Each module creates its own scope. Internal signals and variables are not accessible from outside the module unless explicitly connected through ports.

**Instance vs Module**: A module is the template/definition, while an instance is a specific instantiation of that module in your design.

### Example 1: Simple Counter
simple_counter - Basic module structure with parameters and ANSI-style ports

#### Design under Test (DUT)

In [7]:
# | echo: false

from IPython.display import Markdown, display

# Read SystemVerilog code from file
with open(
    "Chapter_5_examples/example_1__simple_counter/simple_counter.sv",
    "r",
) as source_file:
    sv_code = source_file.read()

display(Markdown(f"```systemverilog\n{sv_code}\n```"))


```systemverilog
// simple_counter.sv
module simple_counter #(
    // Parameters with default values
    parameter int WIDTH = 8,                    // Counter width in bits
    parameter int MAX_COUNT = 255,              // Maximum count value
    parameter bit WRAP_AROUND = 1'b1,           // Enable wrap-around behavior
    parameter int RESET_VALUE = 0               // Reset value for counter
) (
    // ANSI-style port declarations
    input  logic                    clk,        // Clock input
    input  logic                    reset_n,    // Active-low asynchronous reset
    input  logic                    enable,     // Counter enable
    input  logic                    load,       // Load enable
    input  logic [WIDTH-1:0]        load_value, // Value to load
    input  logic                    count_up,   // Count direction (1=up, 0=down)
    output logic [WIDTH-1:0]        count,      // Current counter value
    output logic                    overflow,   // Overflow flag
    output logic                    underflow,  // Underflow flag
    output logic                    max_reached // Maximum count reached flag
);

    // Internal register to hold counter value
    logic [WIDTH-1:0] count_reg;
    
    // Derived parameters using localparam
    localparam logic [WIDTH-1:0] MIN_COUNT = '0;
    localparam logic [WIDTH-1:0] MAX_VAL = MAX_COUNT[WIDTH-1:0];
    localparam logic [WIDTH-1:0] RESET_VAL = RESET_VALUE[WIDTH-1:0];
    
    // Main counter logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            // Asynchronous reset
            count_reg <= RESET_VAL;
            overflow <= 1'b0;
            underflow <= 1'b0;
        end else begin
            // Clear flags by default
            overflow <= 1'b0;
            underflow <= 1'b0;
            
            if (load) begin
                // Load operation has highest priority
                count_reg <= load_value;
            end else if (enable) begin
                if (count_up) begin
                    // Count up logic
                    if (count_reg >= MAX_VAL) begin
                        overflow <= 1'b1;
                        if (WRAP_AROUND) begin
                            count_reg <= MIN_COUNT;
                        end else begin
                            count_reg <= MAX_VAL; // Saturate at maximum
                        end
                    end else begin
                        count_reg <= count_reg + 1'b1;
                    end
                end else begin
                    // Count down logic
                    if (count_reg <= MIN_COUNT) begin
                        underflow <= 1'b1;
                        if (WRAP_AROUND) begin
                            count_reg <= MAX_VAL;
                        end else begin
                            count_reg <= MIN_COUNT; // Saturate at minimum
                        end
                    end else begin
                        count_reg <= count_reg - 1'b1;
                    end
                end
            end
            // If enable is false, counter holds its current value
        end
    end
    
    // Continuous assignments for outputs
    assign count = count_reg;
    assign max_reached = (count_reg == MAX_VAL);
    
    // Assertions for parameter validation (synthesis will ignore these)
    initial begin
        assert (WIDTH > 0) else $error("WIDTH parameter must be greater than 0");
        assert (MAX_COUNT >= 0) else $error("MAX_COUNT parameter must be non-negative");
        assert (MAX_COUNT < (2**WIDTH)) else $error("MAX_COUNT exceeds maximum value for given WIDTH");
        assert (RESET_VALUE >= 0) else $error("RESET_VALUE parameter must be non-negative");
        assert (RESET_VALUE < (2**WIDTH)) else $error("RESET_VALUE exceeds maximum value for given WIDTH");
        
        $display("Simple Counter Module Initialized:");
        $display("  WIDTH = %0d bits", WIDTH);
        $display("  MAX_COUNT = %0d", MAX_COUNT);
        $display("  WRAP_AROUND = %b", WRAP_AROUND);
        $display("  RESET_VALUE = %0d", RESET_VALUE);
        $display("  Counter range: %0d to %0d", MIN_COUNT, MAX_VAL);
    end

endmodule
```

#### Design Unit Test (DUT) Testbench

In [8]:
# | echo: false

from IPython.display import Markdown, display

# Read SystemVerilog code from file
with open(
    "Chapter_5_examples/example_1__simple_counter/simple_counter_testbench.sv",
    "r",
) as source_file:
    sv_code = source_file.read()

display(Markdown(f"```systemverilog\n{sv_code}\n```"))


```systemverilog
// simple_counter_testbench.sv
module simple_counter_testbench;

    // Testbench parameters
    localparam int CLK_PERIOD = 10; // 100MHz clock
    localparam int TEST_WIDTH = 4;   // 4-bit counter for easy testing
    localparam int TEST_MAX = 12;    // Max count less than 2^4-1 for testing
    
    // Testbench signals
    logic                    clk;
    logic                    reset_n;
    logic                    enable;
    logic                    load;
    logic [TEST_WIDTH-1:0]   load_value;
    logic                    count_up;
    logic [TEST_WIDTH-1:0]   count;
    logic                    overflow;
    logic                    underflow;
    logic                    max_reached;
    
    // Clock generation
    initial begin
        clk = 0;
        forever #(CLK_PERIOD/2) clk = ~clk;
    end
    
    // DUT instantiation with custom parameters
    simple_counter #(
        .WIDTH(TEST_WIDTH),
        .MAX_COUNT(TEST_MAX),
        .WRAP_AROUND(1'b1),
        .RESET_VALUE(0)
    ) dut (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .load(load),
        .load_value(load_value),
        .count_up(count_up),
        .count(count),
        .overflow(overflow),
        .underflow(underflow),
        .max_reached(max_reached)
    );
    
    // Second instance with different parameters for comparison
    logic [7:0] count2;
    logic overflow2, underflow2, max_reached2;
    
    simple_counter #(
        .WIDTH(8),
        .MAX_COUNT(255),
        .WRAP_AROUND(1'b0),     // No wrap-around (saturation mode)
        .RESET_VALUE(128)       // Different reset value
    ) dut2 (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable),
        .load(1'b0),            // Disable load for this instance
        .load_value(8'h00),
        .count_up(count_up),
        .count(count2),
        .overflow(overflow2),
        .underflow(underflow2),
        .max_reached(max_reached2)
    );
    
    // Test stimulus
    initial begin
        // Initialize VCD dump
        $dumpfile("simple_counter_testbench.vcd");
        $dumpvars(0, simple_counter_testbench);
        
        $display("=== Simple Counter Testbench Started ===");
        $display();
        
        // Initialize signals
        reset_n = 0;
        enable = 0;
        load = 0;
        load_value = 0;
        count_up = 1;
        
        // Reset phase
        $display("Phase 1: Reset Test");
        #(CLK_PERIOD * 2);
        reset_n = 1;
        #(CLK_PERIOD);
        $display("After reset - DUT1 count: %0d, DUT2 count: %0d", count, count2);
        $display();
        
        // Test counting up
        $display("Phase 2: Count Up Test");
        enable = 1;
        count_up = 1;
        
        repeat (TEST_MAX + 3) begin
            #(CLK_PERIOD);
            $display("Count up - DUT1: %0d (overflow=%b, max_reached=%b), DUT2: %0d (overflow=%b)", 
                    count, overflow, max_reached, count2, overflow2);
        end
        $display();
        
        // Test counting down
        $display("Phase 3: Count Down Test");
        count_up = 0;
        
        repeat (TEST_MAX + 3) begin
            #(CLK_PERIOD);
            $display("Count down - DUT1: %0d (underflow=%b), DUT2: %0d (underflow=%b)", 
                    count, underflow, count2, underflow2);
        end
        $display();
        
        // Test load operation
        $display("Phase 4: Load Operation Test");
        count_up = 1;
        load_value = TEST_WIDTH'(TEST_MAX / 2);  // Explicit width casting
        load = 1;
        #(CLK_PERIOD);
        $display("After load %0d - DUT1 count: %0d", load_value, count);
        load = 0;
        #(CLK_PERIOD);
        $display("After load release - DUT1 count: %0d", count);
        $display();
        
        // Test enable control
        $display("Phase 5: Enable Control Test");
        enable = 0;
        repeat (3) begin
            #(CLK_PERIOD);
            $display("Enable=0 - DUT1 count: %0d (should not change)", count);
        end
        
        enable = 1;
        repeat (3) begin
            #(CLK_PERIOD);
            $display("Enable=1 - DUT1 count: %0d (should increment)", count);
        end
        $display();
        
        // Test different parameter behavior
        $display("Phase 6: Parameter Comparison");
        $display("DUT1 (4-bit, wrap-around): count=%0d, max_count=%0d", count, TEST_MAX);
        $display("DUT2 (8-bit, saturation): count=%0d, max_count=255", count2);
        $display();
        
        // Final phase - reset test
        $display("Phase 7: Final Reset Test");
        reset_n = 0;
        #(CLK_PERIOD);
        reset_n = 1;
        #(CLK_PERIOD);
        $display("After final reset - DUT1: %0d, DUT2: %0d", count, count2);
        $display();
        
        $display("=== Simple Counter Testbench Completed ===");
        $display("Total simulation time: %0t", $time);
        $finish;
    end
    
    // Monitor for detecting important events
    always @(posedge clk) begin
        if (reset_n) begin
            if (overflow)
                $display("*** OVERFLOW detected at time %0t, count=%0d ***", $time, count);
            if (underflow)
                $display("*** UNDERFLOW detected at time %0t, count=%0d ***", $time, count);
            if (max_reached && enable)
                $display("*** MAX_REACHED at time %0t, count=%0d ***", $time, count);
        end
    end

endmodule
```

In [3]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_1__simple_counter/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [6]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_1__simple_counter/")


Docker Compose Output:
 Container notebooks-verilator-1  Recreate
 Container notebooks-verilator-1  Recreated
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[430] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_1__simple_counter/obj_dir directory...
Chapter_5_examples/example_1__simple_counter/obj_dir removed successfully.


0

### Example 2: Data Register
data_register - Module instantiation examples (named vs positional connections)

#### Design under Test (DUT)

In [9]:
# | echo: false

from IPython.display import Markdown, display

# Read SystemVerilog code from file
with open(
    "Chapter_5_examples/example_2__data_register/data_register.sv",
    "r",
) as source_file:
    sv_code = source_file.read()

display(Markdown(f"```systemverilog\n{sv_code}\n```"))


```systemverilog
// data_register.sv
module data_register #(
    // Parameters
    parameter int WIDTH = 8,                    // Data width
    parameter logic [WIDTH-1:0] RESET_VALUE = '0, // Reset value
    parameter bit SYNC_RESET = 1'b0,            // 0=async reset, 1=sync reset
    parameter string REGISTER_NAME = "DATA_REG"  // Register name for debugging
) (
    // Clock and reset
    input  logic                clk,            // Clock input
    input  logic                reset_n,        // Reset (active low)
    
    // Control signals
    input  logic                enable,         // Write enable
    input  logic                load,           // Load enable (higher priority)
    input  logic                clear,          // Clear register
    
    // Data signals
    input  logic [WIDTH-1:0]    data_in,       // Data input
    input  logic [WIDTH-1:0]    load_data,     // Load data input
    output logic [WIDTH-1:0]    data_out,      // Data output
    
    // Status signals
    output logic                valid,          // Data valid flag
    output logic                changed         // Data changed flag
);

    // Internal registers
    logic [WIDTH-1:0] data_reg;
    logic [WIDTH-1:0] prev_data;
    logic valid_reg;
    
    // Generate block for reset type selection
    generate
        if (SYNC_RESET) begin : sync_reset_logic
            // Synchronous reset implementation
            always_ff @(posedge clk) begin
                if (!reset_n) begin
                    data_reg <= RESET_VALUE;
                    prev_data <= RESET_VALUE;
                    valid_reg <= 1'b0;
                end else begin
                    prev_data <= data_reg;  // Store previous value
                    
                    if (clear) begin
                        data_reg <= '0;
                        valid_reg <= 1'b0;
                    end else if (load) begin
                        data_reg <= load_data;
                        valid_reg <= 1'b1;
                    end else if (enable) begin
                        data_reg <= data_in;
                        valid_reg <= 1'b1;
                    end
                end
            end
        end else begin : async_reset_logic
            // Asynchronous reset implementation
            always_ff @(posedge clk or negedge reset_n) begin
                if (!reset_n) begin
                    data_reg <= RESET_VALUE;
                    prev_data <= RESET_VALUE;
                    valid_reg <= 1'b0;
                end else begin
                    prev_data <= data_reg;  // Store previous value
                    
                    if (clear) begin
                        data_reg <= '0;
                        valid_reg <= 1'b0;
                    end else if (load) begin
                        data_reg <= load_data;
                        valid_reg <= 1'b1;
                    end else if (enable) begin
                        data_reg <= data_in;
                        valid_reg <= 1'b1;
                    end
                end
            end
        end
    endgenerate
    
    // Output assignments
    assign data_out = data_reg;
    assign valid = valid_reg;
    assign changed = valid_reg && (data_reg != prev_data);
    
    // Debug display (only in simulation)
    initial begin
        $display("%s: Initialized with WIDTH=%0d, RESET_VALUE=%0h, SYNC_RESET=%b", 
                REGISTER_NAME, WIDTH, RESET_VALUE, SYNC_RESET);
    end
    
    // Monitor changes (simulation only)
    always @(posedge clk) begin
        if (reset_n && changed) begin
            $display("%s: Data changed from %0h to %0h at time %0t", 
                    REGISTER_NAME, prev_data, data_reg, $time);
        end
    end

endmodule
```

In [10]:
# | echo: false

from IPython.display import Markdown, display

# Read SystemVerilog code from file
with open(
    "Chapter_5_examples/example_2__data_register/instantiation_examples.sv",
    "r",
) as source_file:
    sv_code = source_file.read()

display(Markdown(f"```systemverilog\n{sv_code}\n```"))


```systemverilog
// instantiation_examples.sv
module instantiation_examples;

    // Common testbench signals
    logic clk;
    logic reset_n;
    
    // Signals for register instances
    logic enable_1, enable_2, enable_3, enable_4;
    logic load_1, load_2, load_3, load_4;
    logic clear_1, clear_2, clear_3, clear_4;
    logic [7:0] data_in_1, data_in_2, data_in_3, data_in_4;
    logic [7:0] load_data_1, load_data_2, load_data_3, load_data_4;
    logic [7:0] data_out_1, data_out_2, data_out_3, data_out_4;
    logic valid_1, valid_2, valid_3, valid_4;
    logic changed_1, changed_2, changed_3, changed_4;
    
    // Signals for 16-bit register
    logic [15:0] data_in_16, load_data_16, data_out_16;
    logic enable_16, load_16, clear_16, valid_16, changed_16;
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 100MHz clock
    end
    
    //========================================================================
    // EXAMPLE 1: Named Port Connections (RECOMMENDED)
    //========================================================================
    data_register #(
        .WIDTH(8),
        .RESET_VALUE(8'hAA),
        .SYNC_RESET(1'b0),
        .REGISTER_NAME("REG1_NAMED")
    ) reg1_named_inst (
        // Clock and reset
        .clk(clk),
        .reset_n(reset_n),
        
        // Control signals
        .enable(enable_1),
        .load(load_1),
        .clear(clear_1),
        
        // Data signals
        .data_in(data_in_1),
        .load_data(load_data_1),
        .data_out(data_out_1),
        
        // Status signals
        .valid(valid_1),
        .changed(changed_1)
    );
    
    //========================================================================
    // EXAMPLE 2: Named Port Connections with Different Parameters
    //========================================================================
    data_register #(
        .WIDTH(8),
        .RESET_VALUE(8'h55),
        .SYNC_RESET(1'b1),           // Synchronous reset
        .REGISTER_NAME("REG2_SYNC")
    ) reg2_sync_inst (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable_2),
        .load(load_2),
        .clear(clear_2),
        .data_in(data_in_2),
        .load_data(load_data_2),
        .data_out(data_out_2),
        .valid(valid_2),
        .changed(changed_2)
    );
    
    //========================================================================
    // EXAMPLE 3: Named Port Connections with Parameter Override
    //========================================================================
    data_register #(
        .WIDTH(16),                  // 16-bit register
        .RESET_VALUE(16'hDEAD),
        .SYNC_RESET(1'b0),
        .REGISTER_NAME("REG3_16BIT")
    ) reg3_16bit_inst (
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable_16),
        .load(load_16),
        .clear(clear_16),
        .data_in(data_in_16),
        .load_data(load_data_16),
        .data_out(data_out_16),
        .valid(valid_16),
        .changed(changed_16)
    );
    
    //========================================================================
    // EXAMPLE 4: Positional Port Connections (NOT RECOMMENDED)
    // Order must match exactly with module port declaration
    //========================================================================
    data_register #(8, 8'h00, 1'b0, "REG4_POSITIONAL") reg4_positional_inst (
        clk,            // clk
        reset_n,        // reset_n
        enable_3,       // enable
        load_3,         // load
        clear_3,        // clear
        data_in_3,      // data_in
        load_data_3,    // load_data
        data_out_3,     // data_out
        valid_3,        // valid
        changed_3       // changed
    );
    
    //========================================================================
    // EXAMPLE 5: Mixed Named and Positional (NOT RECOMMENDED)
    // This shows what NOT to do - mixing styles is confusing
    //========================================================================
    /* 
    // This would be bad practice - don't mix styles:
    data_register #(.WIDTH(8)) reg_mixed_inst (
        clk, reset_n,           // Positional
        .enable(enable_4),      // Named
        load_4, clear_4,        // Positional again
        .data_in(data_in_4),    // Named again
        // ... this is confusing and error-prone
    );
    */
    
    //========================================================================
    // EXAMPLE 6: Default Parameters with Named Connections
    //========================================================================
    data_register reg5_default_inst (  // Using all default parameters
        .clk(clk),
        .reset_n(reset_n),
        .enable(enable_4),
        .load(load_4),
        .clear(clear_4),
        .data_in(data_in_4),
        .load_data(load_data_4),
        .data_out(data_out_4),
        .valid(valid_4),
        .changed(changed_4)
    );
    
    //========================================================================
    // Test Stimulus
    //========================================================================
    initial begin
        // Initialize VCD dump
        $dumpfile("instantiation_examples.vcd");
        $dumpvars(0, instantiation_examples);
        
        $display("=== Module Instantiation Examples Started ===");
        $display("Demonstrating Named vs Positional Port Connections");
        $display();
        
        // Initialize all signals
        reset_n = 0;
        {enable_1, enable_2, enable_16, enable_3, enable_4} = 5'b00000;
        {load_1, load_2, load_16, load_3, load_4} = 5'b00000;
        {clear_1, clear_2, clear_16, clear_3, clear_4} = 5'b00000;
        data_in_1 = 8'h10;
        data_in_2 = 8'h20;
        data_in_3 = 8'h30;
        data_in_4 = 8'h40;
        data_in_16 = 16'hBEEF;
        load_data_1 = 8'hF1;
        load_data_2 = 8'hF2;
        load_data_3 = 8'hF3;
        load_data_4 = 8'hF4;
        load_data_16 = 16'hCAFE;
        
        // Release reset
        #20 reset_n = 1;
        $display("Reset released - checking initial values:");
        #10;
        $display("REG1 (named, async): data_out=%02h, valid=%b", data_out_1, valid_1);
        $display("REG2 (named, sync):  data_out=%02h, valid=%b", data_out_2, valid_2);
        $display("REG3 (16-bit):       data_out=%04h, valid=%b", data_out_16, valid_16);
        $display("REG4 (positional):   data_out=%02h, valid=%b", data_out_3, valid_3);
        $display("REG5 (default):      data_out=%02h, valid=%b", data_out_4, valid_4);
        $display();
        
        // Test normal write operations
        $display("Testing normal write operations:");
        enable_1 = 1; enable_2 = 1; enable_16 = 1; enable_3 = 1; enable_4 = 1;
        #10;
        $display("After enable - all registers should have new data:");
        $display("REG1: %02h, REG2: %02h, REG3: %04h, REG4: %02h, REG5: %02h", 
                data_out_1, data_out_2, data_out_16, data_out_3, data_out_4);
        
        // Test load operations (higher priority)
        $display();
        $display("Testing load operations (higher priority than enable):");
        load_1 = 1; load_2 = 1; load_16 = 1; load_3 = 1; load_4 = 1;
        #10;
        $display("After load - registers should have load_data values:");
        $display("REG1: %02h, REG2: %02h, REG3: %04h, REG4: %02h, REG5: %02h", 
                data_out_1, data_out_2, data_out_16, data_out_3, data_out_4);
        
        load_1 = 0; load_2 = 0; load_16 = 0; load_3 = 0; load_4 = 0;
        
        // Test clear operations
        $display();
        $display("Testing clear operations:");
        clear_1 = 1; clear_2 = 1; clear_16 = 1; clear_3 = 1; clear_4 = 1;
        #10;
        $display("After clear - all registers should be zero:");
        $display("REG1: %02h, REG2: %02h, REG3: %04h, REG4: %02h, REG5: %02h", 
                data_out_1, data_out_2, data_out_16, data_out_3, data_out_4);
        
        clear_1 = 0; clear_2 = 0; clear_16 = 0; clear_3 = 0; clear_4 = 0;
        enable_1 = 0; enable_2 = 0; enable_16 = 0; enable_3 = 0; enable_4 = 0;
        
        #50;
        $display();
        $display("=== Instantiation Examples Summary ===");
        $display("1. Named connections (RECOMMENDED):");
        $display("   - Clear and readable");
        $display("   - Order independent");
        $display("   - Less error-prone");
        $display("   - Easy to maintain");
        $display();
        $display("2. Positional connections (NOT RECOMMENDED):");
        $display("   - Order dependent");
        $display("   - Error-prone");
        $display("   - Hard to maintain");
        $display("   - Only use for very simple modules");
        $display();
        $display("3. Parameter override examples shown:");
        $display("   - Different widths");
        $display("   - Different reset types");
        $display("   - Different reset values");
        $display("   - Using default parameters");
        $display();
        
        $finish;
    end

endmodule
```

#### Design Unit Test (DUT) Testbench

In [11]:
# | echo: false

from IPython.display import Markdown, display

# Read SystemVerilog code from file
with open(
    "Chapter_5_examples/example_2__data_register/data_register_testbench.sv",
    "r",
) as source_file:
    sv_code = source_file.read()

display(Markdown(f"```systemverilog\n{sv_code}\n```"))


```systemverilog
// data_register_testbench.sv
module data_register_testbench;

    // Instantiate the examples module
    instantiation_examples inst_examples();
    
    initial begin
        // Set up VCD dumping
        $dumpfile("data_register_testbench.vcd");
        $dumpvars(0, data_register_testbench);
        
        $display();
        $display("=== Data Register Testbench ===");
        $display("This testbench demonstrates different module instantiation styles:");
        $display("- Named port connections (recommended)");
        $display("- Positional port connections (not recommended)");
        $display("- Parameter overrides");
        $display("- Mixed parameter and port connection styles");
        $display();
        
        // Wait for the examples to complete
        #1000;
        
        $display("=== Key Takeaways ===");
        $display();
        $display("BEST PRACTICES:");
        $display("âœ“ Always use named port connections");
        $display("âœ“ Use explicit parameter names when overriding");
        $display("âœ“ Group related signals logically");
        $display("âœ“ Use consistent indentation and formatting");
        $display();
        $display("AVOID:");
        $display("âœ— Positional port connections (except for very simple modules)");
        $display("âœ— Mixing named and positional styles");
        $display("âœ— Long parameter lists without names");
        $display("âœ— Inconsistent formatting");
        $display();
        $display("PARAMETER OVERRIDE EXAMPLES:");
        $display("- data_register #(.WIDTH(16)) inst_name (...)");
        $display("- data_register #(.WIDTH(8), .SYNC_RESET(1)) inst_name (...)");
        $display("- data_register inst_name (...) // Uses all defaults");
        $display();
        
        $finish;
    end

endmodule
```

In [4]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_2__data_register/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [7]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_2__data_register/")


Docker Compose Output:
 Container notebooks-verilator-1  Recreate
 Container notebooks-verilator-1  Recreated
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[110] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_2__data_register/obj_dir directory...
Chapter_5_examples/example_2__data_register/obj_dir removed successfully.


0

## Port Declarations and Directions

SystemVerilog provides several ways to declare module ports, offering more flexibility than traditional Verilog.

### Port Directions

```systemverilog
module port_example (
    input  logic        clk,           // Input port
    output logic        valid,         // Output port
    inout  wire         bidir_signal,  // Bidirectional port
    ref    int          shared_var     // Reference port (SystemVerilog)
);
```

### ANSI-Style Port Declarations (Recommended)

```systemverilog
module counter #(
    parameter int WIDTH = 8
) (
    input  logic             clk,
    input  logic             reset_n,
    input  logic             enable,
    input  logic             load,
    input  logic [WIDTH-1:0] load_value,
    output logic [WIDTH-1:0] count,
    output logic             overflow
);

    logic [WIDTH-1:0] count_reg;
    
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            count_reg <= '0;
            overflow <= 1'b0;
        end else if (load) begin
            count_reg <= load_value;
            overflow <= 1'b0;
        end else if (enable) begin
            {overflow, count_reg} <= count_reg + 1'b1;
        end
    end
    
    assign count = count_reg;

endmodule
```

### Non-ANSI Style (Legacy)

```systemverilog
module counter (clk, reset_n, enable, count);
    parameter WIDTH = 8;
    
    input             clk;
    input             reset_n;
    input             enable;
    output [WIDTH-1:0] count;
    
    // Port declarations separate from module header
endmodule
```

### Advanced Port Features

**Interface Ports**:
```systemverilog
module processor (
    input logic clk,
    input logic reset_n,
    memory_if.master mem_bus,  // Interface port
    axi4_if.slave    axi_port
);
```

**Unpacked Array Ports**:
```systemverilog
module multi_port (
    input  logic [7:0] data_in [0:3],   // Array of inputs
    output logic [7:0] data_out [0:3]   // Array of outputs
);
```

### Example 3: Port Direction Demo
port_direction_demo - Different port types (input, output, inout, ref)

In [6]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_3__port_directions/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [8]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_3__port_directions/")


Docker Compose Output:
 Container notebooks-verilator-1  Recreate
 Container notebooks-verilator-1  Recreated
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[230] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_3__port_directions/obj_dir directory...
Chapter_5_examples/example_3__port_directions/obj_dir removed successfully.


0

### Example 4: Array Port Module
array_port_module - Unpacked array ports demonstration

In [9]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_4__array_port_module/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [9]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_4__array_port_module/")


Docker Compose Output:
 Container notebooks-verilator-1  Recreate
 Container notebooks-verilator-1  Recreated
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[270] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_4__array_port_module/obj_dir directory...
Chapter_5_examples/example_4__array_port_module/obj_dir removed successfully.


0

## Parameters and Localparams

Parameters provide a way to create configurable, reusable modules. They allow customization at instantiation time.

### Parameter Types

```systemverilog
module parameterized_module #(
    // Type parameters
    parameter type DATA_TYPE = logic [31:0],
    parameter type ADDR_TYPE = logic [15:0],
    
    // Value parameters
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16,
    parameter int DEPTH = 1024,
    
    // String parameters
    parameter string MODE = "NORMAL",
    
    // Real parameters
    parameter real FREQUENCY = 100.0
) (
    input  logic      clk,
    input  DATA_TYPE  data_in,
    input  ADDR_TYPE  address,
    output DATA_TYPE  data_out
);
```

### Localparam Usage

Localparams are parameters that cannot be overridden during instantiation. They're typically used for derived values.

```systemverilog
module memory #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 10
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    // Localparams derived from parameters
    localparam int DEPTH = 2**ADDR_WIDTH;
    localparam int BYTES_PER_WORD = DATA_WIDTH / 8;
    
    logic [DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge clk) begin
        if (we)
            mem_array[addr] <= wdata;
        rdata <= mem_array[addr];
    end

endmodule
```

### Parameter Override Examples

```systemverilog
// Override during instantiation
memory #(
    .DATA_WIDTH(64),
    .ADDR_WIDTH(12)
) ram_inst (
    .clk(clk),
    .we(write_enable),
    .addr(address),
    .wdata(write_data),
    .rdata(read_data)
);

// Using defparam (not recommended)
defparam ram_inst.DATA_WIDTH = 64;
defparam ram_inst.ADDR_WIDTH = 12;
```

### Example 5: Configurable Memory
configurable_memory - Parameter types (type, value, string, real parameters)

In [5]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_5__configurable_memory/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [10]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_5__configurable_memory/")


Docker Compose Output:
 Container notebooks-verilator-1  Recreate
 Container notebooks-verilator-1  Recreated
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[780] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_5__configurable_memory/obj_dir directory...
Chapter_5_examples/example_5__configurable_memory/obj_dir removed successfully.


0

### Example 6: Parameter Override
parameter_override - Parameter override during instantiation

In [7]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_6__parameter_override/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

In [2]:
from gtkwave_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_6__parameter_override/")


Docker Compose Output:
 Container notebooks-verilator-1  Starting
 Container notebooks-verilator-1  Started

GTKWave Analyzer v3.3.104 (w)1999-2020 BSI

[0] start time.
[426] end time.
WM Destroy
Process finished with return code: 0
Removing Chapter_5_examples/example_6__parameter_override/obj_dir directory...
Chapter_5_examples/example_6__parameter_override/obj_dir removed successfully.


0

### Example 7: Localparam Calculator
localparam_calculator - Localparam usage for derived values

In [1]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_7__localparam_calculator/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

## Generate Blocks

Generate blocks allow you to create repetitive hardware structures and conditional compilation based on parameters.

### Generate For Loops

```systemverilog
module parallel_adder #(
    parameter int WIDTH = 32,
    parameter int STAGES = 4
) (
    input  logic [WIDTH-1:0] a,
    input  logic [WIDTH-1:0] b,
    input  logic             cin,
    output logic [WIDTH-1:0] sum,
    output logic             cout
);

    localparam int BITS_PER_STAGE = WIDTH / STAGES;
    
    logic [STAGES:0] carry;
    assign carry[0] = cin;
    assign cout = carry[STAGES];
    
    // Generate multiple adder stages
    generate
        for (genvar i = 0; i < STAGES; i++) begin : adder_stage
            logic [BITS_PER_STAGE-1:0] stage_sum;
            logic                      stage_cout;
            
            full_adder #(.WIDTH(BITS_PER_STAGE)) fa_inst (
                .a(a[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .b(b[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .cin(carry[i]),
                .sum(stage_sum),
                .cout(stage_cout)
            );
            
            assign sum[i*BITS_PER_STAGE +: BITS_PER_STAGE] = stage_sum;
            assign carry[i+1] = stage_cout;
        end
    endgenerate

endmodule
```

### Generate If-Else

```systemverilog
module configurable_memory #(
    parameter int    DATA_WIDTH = 32,
    parameter int    ADDR_WIDTH = 10,
    parameter string MEMORY_TYPE = "BLOCK"  // "BLOCK" or "DISTRIBUTED"
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    localparam int DEPTH = 2**ADDR_WIDTH;
    
    generate
        if (MEMORY_TYPE == "BLOCK") begin : block_memory
            // Use block RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
                rdata <= mem[addr];
            end
            
        end else if (MEMORY_TYPE == "DISTRIBUTED") begin : dist_memory
            // Use distributed RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
            end
            
            assign rdata = mem[addr];  // Combinational read
            
        end else begin : error_memory
            // Generate compile-time error for invalid parameter
            initial begin
                $error("Invalid MEMORY_TYPE parameter: %s", MEMORY_TYPE);
            end
        end
    endgenerate

endmodule
```

### Generate Case

```systemverilog
module priority_encoder #(
    parameter int WIDTH = 8
) (
    input  logic [WIDTH-1:0] data_in,
    output logic [$clog2(WIDTH)-1:0] encoded_out,
    output logic valid
);

    generate
        case (WIDTH)
            4: begin : enc_4bit
                always_comb begin
                    casez (data_in)
                        4'b???1: {valid, encoded_out} = {1'b1, 2'd0};
                        4'b??10: {valid, encoded_out} = {1'b1, 2'd1};
                        4'b?100: {valid, encoded_out} = {1'b1, 2'd2};
                        4'b1000: {valid, encoded_out} = {1'b1, 2'd3};
                        default: {valid, encoded_out} = {1'b0, 2'd0};
                    endcase
                end
            end
            
            8: begin : enc_8bit
                // Implementation for 8-bit encoder
                always_comb begin
                    casez (data_in)
                        8'b???????1: {valid, encoded_out} = {1'b1, 3'd0};
                        8'b??????10: {valid, encoded_out} = {1'b1, 3'd1};
                        8'b?????100: {valid, encoded_out} = {1'b1, 3'd2};
                        8'b????1000: {valid, encoded_out} = {1'b1, 3'd3};
                        8'b???10000: {valid, encoded_out} = {1'b1, 3'd4};
                        8'b??100000: {valid, encoded_out} = {1'b1, 3'd5};
                        8'b?1000000: {valid, encoded_out} = {1'b1, 3'd6};
                        8'b10000000: {valid, encoded_out} = {1'b1, 3'd7};
                        default:     {valid, encoded_out} = {1'b0, 3'd0};
                    endcase
                end
            end
            
            default: begin : enc_generic
                // Generic implementation for other widths
                always_comb begin
                    encoded_out = '0;
                    valid = 1'b0;
                    for (int i = 0; i < WIDTH; i++) begin
                        if (data_in[i]) begin
                            encoded_out = i[$clog2(WIDTH)-1:0];
                            valid = 1'b1;
                            break;
                        end
                    end
                end
            end
        endcase
    endgenerate

endmodule
```

### Example 8: Parallel Adder Generator
parallel_adder_generator - Generate for loops creating repetitive structures

In [2]:
from verilator_runner import run_docker_compose

run_docker_compose("Chapter_5_examples/example_8__parallel_adder_generator/")


Docker Compose Output:
make: Entering directory '/work/obj_dir'
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized
-Wno-unused-but-set-parameter -Wno-unused-but-set-variable -Wno-unused-parameter
-Wno-unused-variable    -DVL_TIME_CONTEXT   -fcoroutines -c -o verilated.o
/usr/local/share/verilator/include/verilated.cpp
ccache g++ -Os  -I.  -MMD -I/usr/local/share/verilator/include
-I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0
-DVM_TIMING=1 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -DVM_TRACE_SAIF=0
-faligned-new -fcf-protection=none -Wno-bool-operation -Wno-shadow -Wno-sign-
compare -Wno-subobject-linkage -Wno-tautological-compare -Wno-uninitialized

0

### Example 9: Memory Type Selector
memory_type_selector - Generate if-else for conditional compilation

### Example 10: Encoder Width Selector
encoder_width_selector - Generate case for parameter-based selection

## Introduction to Interfaces

Interfaces provide a powerful way to group related signals and simplify connections between modules. They help reduce port lists and improve code maintainability.

### Basic Interface Declaration

```systemverilog
interface memory_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16
) (
    input logic clk,
    input logic reset_n
);

    // Interface signals
    logic                    valid;
    logic                    ready;
    logic                    we;
    logic [ADDR_WIDTH-1:0]   addr;
    logic [DATA_WIDTH-1:0]   wdata;
    logic [DATA_WIDTH-1:0]   rdata;
    logic                    error;
    
    // Tasks and functions can be defined in interfaces
    task write_transaction(
        input logic [ADDR_WIDTH-1:0] address,
        input logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b1;
        addr <= address;
        wdata <= data;
        @(posedge clk);
        while (!ready) @(posedge clk);
        valid <= 1'b0;
        we <= 1'b0;
    endtask
    
    task read_transaction(
        input  logic [ADDR_WIDTH-1:0] address,
        output logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b0;
        addr <= address;
        @(posedge clk);
        while (!ready) @(posedge clk);
        data = rdata;
        valid <= 1'b0;
    endtask

endinterface
```

### Using Interfaces in Modules

```systemverilog
// Memory controller module
module memory_controller (
    memory_if.slave  cpu_if,    // CPU interface (slave perspective)
    memory_if.master mem_if     // Memory interface (master perspective)
);

    // Interface connection logic
    always_comb begin
        // Forward CPU requests to memory
        mem_if.valid = cpu_if.valid;
        mem_if.we    = cpu_if.we;
        mem_if.addr  = cpu_if.addr;
        mem_if.wdata = cpu_if.wdata;
        
        // Forward memory responses to CPU
        cpu_if.ready = mem_if.ready;
        cpu_if.rdata = mem_if.rdata;
        cpu_if.error = mem_if.error;
    end

endmodule

// Memory module
module memory (
    memory_if.slave mem_if
);

    localparam int DEPTH = 2**mem_if.ADDR_WIDTH;
    logic [mem_if.DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge mem_if.clk or negedge mem_if.reset_n) begin
        if (!mem_if.reset_n) begin
            mem_if.ready <= 1'b0;
            mem_if.rdata <= '0;
            mem_if.error <= 1'b0;
        end else begin
            mem_if.ready <= mem_if.valid;
            mem_if.error <= 1'b0;
            
            if (mem_if.valid) begin
                if (mem_if.we) begin
                    mem_array[mem_if.addr] <= mem_if.wdata;
                end else begin
                    mem_if.rdata <= mem_array[mem_if.addr];
                end
            end
        end
    end

endmodule
```

### Interface Instantiation and Connection

```systemverilog
module top_level;
    logic clk, reset_n;
    
    // Interface instances
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) cpu_mem_if(clk, reset_n);
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) ctrl_mem_if(clk, reset_n);
    
    // Module instances
    cpu cpu_inst (
        .clk(clk),
        .reset_n(reset_n),
        .mem_if(cpu_mem_if.master)  // CPU is master
    );
    
    memory_controller ctrl_inst (
        .cpu_if(cpu_mem_if.slave),   // Controller is slave to CPU
        .mem_if(ctrl_mem_if.master)  // Controller is master to memory
    );
    
    memory mem_inst (
        .mem_if(ctrl_mem_if.slave)   // Memory is slave
    );

endmodule
```

### Example 11: Basic Memory Interface
basic_memory_interface - Simple interface declaration and usage

### Example 12: Basic Memory Interface
interface_with_tasks - Interface with embedded tasks and functions

## Modports and Clocking Blocks

Modports define different views of an interface for different modules, while clocking blocks provide synchronous timing control.

### Modports

Modports specify which signals are inputs, outputs, or inouts from a particular module's perspective.

```systemverilog
interface axi4_lite_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 32
) (
    input logic aclk,
    input logic aresetn
);

    // Write Address Channel
    logic [ADDR_WIDTH-1:0]  awaddr;
    logic [2:0]             awprot;
    logic                   awvalid;
    logic                   awready;
    
    // Write Data Channel
    logic [DATA_WIDTH-1:0]  wdata;
    logic [(DATA_WIDTH/8)-1:0] wstrb;
    logic                   wvalid;
    logic                   wready;
    
    // Write Response Channel
    logic [1:0]             bresp;
    logic                   bvalid;
    logic                   bready;
    
    // Read Address Channel
    logic [ADDR_WIDTH-1:0]  araddr;
    logic [2:0]             arprot;
    logic                   arvalid;
    logic                   arready;
    
    // Read Data Channel
    logic [DATA_WIDTH-1:0]  rdata;
    logic [1:0]             rresp;
    logic                   rvalid;
    logic                   rready;
    
    // Master modport (drives address/data, receives responses)
    modport master (
        input  aclk, aresetn,
        output awaddr, awprot, awvalid,
        input  awready,
        output wdata, wstrb, wvalid,
        input  wready,
        input  bresp, bvalid,
        output bready,
        output araddr, arprot, arvalid,
        input  arready,
        input  rdata, rresp, rvalid,
        output rready
    );
    
    // Slave modport (receives address/data, drives responses)
    modport slave (
        input  aclk, aresetn,
        input  awaddr, awprot, awvalid,
        output awready,
        input  wdata, wstrb, wvalid,
        output wready,
        output bresp, bvalid,
        input  bready,
        input  araddr, arprot, arvalid,
        output arready,
        output rdata, rresp, rvalid,
        input  rready
    );
    
    // Monitor modport (all inputs for verification)
    modport monitor (
        input aclk, aresetn,
        input awaddr, awprot, awvalid, awready,
        input wdata, wstrb, wvalid, wready,
        input bresp, bvalid, bready,
        input araddr, arprot, arvalid, arready,
        input rdata, rresp, rvalid, rready
    );

endinterface
```

### Clocking Blocks

Clocking blocks define synchronous timing relationships and provide a clean way to handle clocked signals in testbenches.

```systemverilog
interface processor_if (
    input logic clk,
    input logic reset_n
);

    logic [31:0] instruction;
    logic [31:0] pc;
    logic        valid;
    logic        ready;
    logic        stall;
    logic        flush;
    
    // Clocking block for testbench use
    clocking cb @(posedge clk);
        default input #1step output #2ns;  // Input skew and output delay
        
        input  pc, valid, ready;
        output instruction, stall, flush;
    endclocking
    
    // Separate clocking block for different timing requirements
    clocking slow_cb @(posedge clk);
        default input #5ns output #10ns;
        
        input  pc, valid;
        output instruction;
    endclocking
    
    // Modports with clocking blocks
    modport tb (
        clocking cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        output pc, valid, ready,
        input  instruction, stall, flush
    );

endinterface
```

### Advanced Clocking Block Example

```systemverilog
interface memory_test_if (
    input logic clk,
    input logic reset_n
);

    logic [15:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic        we;
    logic        re;
    logic        valid;
    logic        ready;
    
    // Clocking block with different timing for different signals
    clocking driver_cb @(posedge clk);
        default input #2ns output #1ns;
        
        output addr, wdata, we, re, valid;
        input  rdata, ready;
    endclocking
    
    // Monitor clocking block samples everything
    clocking monitor_cb @(posedge clk);
        default input #1step;
        
        input addr, wdata, rdata, we, re, valid, ready;
    endclocking
    
    // Synchronous reset clocking block
    clocking reset_cb @(posedge clk);
        input reset_n;
    endclocking
    
    modport driver (
        clocking driver_cb,
        input clk, reset_n
    );
    
    modport monitor (
        clocking monitor_cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        input  addr, wdata, we, re, valid,
        output rdata, ready
    );

endinterface
```

### Using Clocking Blocks in Testbenches

```systemverilog
module memory_testbench;
    logic clk = 0;
    logic reset_n;
    
    always #5ns clk = ~clk;  // 100MHz clock
    
    memory_test_if mem_if(clk, reset_n);
    
    // DUT instantiation
    memory dut (
        .mem_if(mem_if.dut)
    );
    
    // Test program using clocking blocks
    initial begin
        reset_n = 0;
        ##2 reset_n = 1;  // Wait 2 clock cycles
        
        // Write operation using clocking block
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.wdata <= 32'hDEADBEEF;
        mem_if.driver_cb.we    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;  // Wait 1 clock cycle
        
        wait (mem_if.driver_cb.ready);  // Wait for ready
        
        mem_if.driver_cb.we    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##2;  // Wait before read
        
        // Read operation
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.re    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;
        
        wait (mem_if.driver_cb.ready);
        
        $display("Read data: %h", mem_if.driver_cb.rdata);
        
        mem_if.driver_cb.re    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##5;
        $finish;
    end

endmodule
```

axi4_lite_interface - Complete interface with master/slave/monitor modports

clocked_processor_interface - Clocking blocks with different timing requirements

testbench_with_clocking - Using clocking blocks in verification environment

## Summary

This chapter covered the essential concepts of SystemVerilog modules and interfaces:

**Modules** form the basic building blocks with proper port declarations and hierarchical instantiation capabilities.

**Parameters and localparams** enable configurable and reusable designs with type safety and parameter validation.

**Generate blocks** provide powerful compile-time code generation for creating repetitive structures and conditional compilation.

**Interfaces** simplify complex designs by grouping related signals and providing reusable communication protocols.

**Modports** define different perspectives of interfaces for various modules, ensuring proper signal direction and access control.

**Clocking blocks** provide precise timing control for synchronous designs, particularly useful in verification environments.

These features work together to create scalable, maintainable, and reusable SystemVerilog designs that can handle complex digital systems efficiently.